In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# NPBのページURL
url = "https://npb.jp/bis/teams/"

# HTMLを取得
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = response.apparent_encoding  # 文字化け防止
soup = BeautifulSoup(response.text, "html.parser")

# データを格納するリスト
teams_data = []

# `<h4>` タグ（リーグ名）を取得
for h4_tag in soup.find_all("h4"):
    league_name = h4_tag.text.strip()  # リーグ名（例: CENTRAL LEAGUE）

    # `<h4>` タグの次の `<ul>` 内のチーム情報を取得
    ul_tag = h4_tag.find_next("ul")
    if ul_tag:
        for a_tag in ul_tag.find_all("a", href=True):
            team_name = a_tag.text.strip()  # チーム名
            team_url = a_tag["href"]  # チームページURL（相対URL）

            # ロゴ画像 (`.png` を含む `<img>` タグを探す)
            img_tag = a_tag.find("img", src=True)
            logo_url = img_tag["src"] if img_tag and ".png" in img_tag["src"] else None

            # 相対URLを絶対URLに変換
            if not team_url.startswith("http"):
                team_url = f"https://npb.jp/bis/teams/{team_url}"
            if logo_url and not logo_url.startswith("http"):
                logo_url = f"https:{logo_url}"

            # データをリストに追加
            teams_data.append([league_name, team_name, logo_url, team_url])

# pandasでデータフレーム作成
df_team = pd.DataFrame(teams_data, columns=["リーグ名", "チーム名", "ロゴURL", "チームページURL"])

df_team.head(20)

,リーグ名,チーム名,ロゴURL,チームページURL
0,CENTRAL LEAGUE,阪神タイガース,https://p.npb.jp/img/common/logo/2024/logo_t_m...,https://npb.jp/bis/teams/rst_t.html
1,CENTRAL LEAGUE,広島東洋カープ,https://p.npb.jp/img/common/logo/2024/logo_c_m...,https://npb.jp/bis/teams/rst_c.html
2,CENTRAL LEAGUE,横浜DeNAベイスターズ,https://p.npb.jp/img/common/logo/2024/logo_db_...,https://npb.jp/bis/teams/rst_db.html
3,CENTRAL LEAGUE,読売ジャイアンツ,https://p.npb.jp/img/common/logo/2024/logo_g_m...,https://npb.jp/bis/teams/rst_g.html
4,CENTRAL LEAGUE,東京ヤクルトスワローズ,https://p.npb.jp/img/common/logo/2024/logo_s_m...,https://npb.jp/bis/teams/rst_s.html
5,CENTRAL LEAGUE,中日ドラゴンズ,https://p.npb.jp/img/common/logo/2024/logo_d_m...,https://npb.jp/bis/teams/rst_d.html
6,PACIFIC LEAGUE,オリックス・バファローズ,https://p.npb.jp/img/common/logo/2024/logo_b_m...,https://npb.jp/bis/teams/rst_b.html
7,PACIFIC LEAGUE,千葉ロッテマリーンズ,https://p.npb.jp/img/common/logo/2024/logo_m_m...,https://npb.jp/bis/teams/rst_m.html
8,PACIFIC LEAGUE,福岡ソフトバンクホークス,https://p.npb.jp/img/common/logo/2024/logo_h_m...,https://npb.jp/bis/teams/rst_h.html
9,PACIFIC LEAGUE,東北楽天ゴールデンイーグルス,https://p.npb.jp/img/common/logo/2024/logo_e_m...,https://npb.jp/bis/teams/rst_e.html


In [20]:
data = []

for team_name, url in zip(df["チーム名"], df["チームページURL"]):
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # エラーチェック
    
    # 自動エンコーディング適用
    response.encoding = response.apparent_encoding

    # HTMLを解析
    soup = BeautifulSoup(response.text, "html.parser")

    # 選手一覧の取得 (tr class="rosterPlayer")
    players = soup.find_all("tr", class_="rosterPlayer")

    # 各選手の情報を取得
    for player in players:
        columns = player.find_all("td")  # 各列（tdタグ）
        if len(columns) >= 5:  # 必要な情報が入っているか確認
            number = columns[0].text.strip()  # 背番号
            name = columns[1].text.strip()    # 選手名
            height = columns[3].text.strip()  # 身長
            weight = columns[4].text.strip()  # 体重
            data.append([team_name, number, name, height, weight])

df_players = pd.DataFrame(data, columns=["team", "number", "name", "height", "weight"])
df_players.head(10)

,team,number,name,height,weight
0,阪神タイガース,80,岡田 彰布,,
1,阪神タイガース,00,ゲラ,183,86
2,阪神タイガース,13,岩崎 優,185,91
3,阪神タイガース,14,岩貞 祐太,183,87
4,阪神タイガース,15,西 純矢,184,94
5,阪神タイガース,16,西 勇輝,181,82
6,阪神タイガース,17,青柳 晃洋,183,83
7,阪神タイガース,19,下村 海翔,174,73
8,阪神タイガース,20,森木 大智,184,93
9,阪神タイガース,21,秋山 拓巳,188,101


In [21]:
# JSONに変換（辞書形式）
json_data = df_players.to_json(orient="records", force_ascii=False)

# ファイル保存（iOSアプリに渡す）
with open("players.json", "w", encoding="utf-8") as f:
    f.write(json_data)